### The Battle of Neighborhoods (Week 1)

<h5> A description of the problem and a discussion of the background </h5>

Bengaluru (old name Bangalore) is IT hub of India. College grads migrate from different states to start their IT career in Bengaluru. 50% population of this city is migrant. Bengaluru comes in top 10 Indian cities for multilingual people. Because of massive development of IT industry in this city, there are other industries and businesses growing here like real state, hotel/hospitality industry, restaurants,e-Commerce and retail industry. Because of the diversity, different type of people like to spend their weekends and holidays differently, some people like visit restaurants and bars,some people like to spend time with family and kids in shopping malls and some people like to go for trips. If someone is looking to open a new business, a restaurant or superstore, s/he would need to understand population density of different neighborhoods of Bengaluru and already existing restaurants, bars and superstores.

<h5>A description of the data and how it will be used to solve the problem</h5>

I would collect population density of different neighborhood of Bengaluru. Using latitude and longitude for the neighborhood, I would explore them for restaurants/bars/superstore/malls with the help of Foursquare APIs.  
Higher population density with less number of restaurants/bars/superstore/malls would be the best places to start a new business.



### Data Source

Bruhat Bengaluru Mahanagara Palike official website<br>

http://bbmp.gov.in/documents/10180/460906/BBMPR_ward_master_BBMP+Restructuring+03-08-2015.pdf

In [46]:
# The code was removed by Watson Studio for sharing.

,Ward_Name,Population_density_(persons_per_sq_km)
0,Kempegowda,3182
1,Chowdeswari,5635
2,Atturu,6606
3,Yelahanka Satellite Town,9224
4,Jakkuru,2215


In [67]:
df_data.shape

(198, 2)

In [28]:
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geopy-1.21.0               |             py_0          58 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.21.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [68]:
def getBlrData(ward_name,population_density):
    data=[]
    for name,density in zip (ward_name,population_density):
        row=[]
        address = name +', Bangalore, India'
        geolocator = Nominatim(user_agent="ny_explorer")
        location = geolocator.geocode(address)
        if location is not None:
            lat = location.latitude
            long = location.longitude
            row.append(name)
            row.append(density)
            row.append(lat)
            row.append(long)
        data.append(row)
    return(data)

In [69]:
blr_data=getBlrData(ward_name=df_data['Ward_Name'],
                    population_density=df_data['Population_density_(persons_per_sq_km)'])

In [83]:
Bangalore_df=pd.DataFrame(blr_data,columns=['Neighbourhood','Population_density_(persons_per_sq_km)','Latitude','Longitude'])
Bangalore_df.dropna(how='any',inplace=True)
Bangalore_df['Population_density_(persons_per_sq_km)']=Bangalore_df['Population_density_(persons_per_sq_km)'].astype(int)
Bangalore_df.head()

,Neighbourhood,Population_density_(persons_per_sq_km),Latitude,Longitude
0,Kempegowda,3182,13.109018,77.601900
1,Chowdeswari,5635,13.120459,77.579618
2,Atturu,6606,13.100409,77.858725
3,Yelahanka Satellite Town,9224,13.089139,77.582717
4,Jakkuru,2215,13.098320,77.625146


In [73]:
Bangalore_df.shape

(168, 4)

In [74]:
address = 'Bangalore, India'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
blr_lat = location.latitude
blr_long = location.longitude
print('The geograpical coordinate of Bangalore are {}, {}.'.format(blr_lat, blr_long))

The geograpical coordinate of Bangalore are 12.9791198, 77.5912997.


In [86]:
import folium
map_blr = folium.Map(location=[blr_lat, blr_long], zoom_start=10)
# add markers to map
for lat, lng, label,density in zip(Bangalore_df['Latitude'], Bangalore_df['Longitude'], Bangalore_df['Neighbourhood'],
                           Bangalore_df['Population_density_(persons_per_sq_km)']):
    label = folium.Popup(label+','+ str(density), parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='yellow',
        fill_opacity=0.7,
        parse_html=False).add_to(map_blr) 
map_blr